# Convolutional Neural Network 모델 정의

In [ ]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# Conv2d 생성
layer = nn.Conv2d(
    in_channels=3,  # 입력 데이터의 channel 개수. 입력 tensor의 shape: (batch_size, channel, height, width) 
    out_channels=5, # 필터의 개수 (output feature map의 개수)
    kernel_size=3,  # 필터의 크기 (3, 3)
    stride=1,       # 계산을 위하 이동 크기. 좌->우: 1칸씩, 상->하: 1칸 (default: 1)
    padding=1,      # 패팅 크기 (정수: 상하/좌우 동일할 패팅크기를 명시 - 0(default): 패딩추가 안함.)
                    # "same": 입력 size와 동일한 size의 출력이 나오도록 알아서 패딩을 추가.
)
# 다음 layer 생성
# layer2 = nn.Conv2d(
#     in_channels=5, # 이전 Conv2d의 out_channels 값이 다음 Conv2d의 in_channels 값이 된다.
# )

In [ ]:
input_data = torch.ones(1, 3, 10, 10) # batch크기, channel수, heigth, width
output = layer(input_data)
output.shape

In [ ]:
### Conv2d의 weigth 의 shape
layer.weight.shape
# [5:필터개수-out_channels, 
#  3:channel수-in_channel, 
#  3:height-kernel_size, 
#  3:width-kernel_size]

In [ ]:
layer.bias.shape # channel당 1개씩 bias가 추가.

In [ ]:
pool_layer = nn.MaxPool2d(
    kernel_size=2, # 값을 추출하는 영역 크기(2, 2) - default: 2
    stride=2,      # 다음 값을 추출하기위해서 몇칸을 이동할지.(default: kernel_size)
    padding=0
    # 값을 추출할 영역이 kernel_size보다 작을 경우 추출할지 여부.
    # 0-추출을 하지 않겠다.
)

In [ ]:
print(input_data.shape)
pool_output = pool_layer(input_data)
print(pool_output.shape)

In [ ]:
input_data = torch.randn(1, 1, 5, 5)

print(input_data.shape)
pool_output = pool_layer(input_data)
print(pool_output.shape)

In [ ]:
input_data[0, 0]

In [ ]:
pool_output

# MNIST

In [ ]:
%pip install torchinfo

In [ ]:
import os

import torch
from torch import nn
import torchinfo

import matplotlib.pyplot as plt
import numpy as np

from module.data import load_mnist_dataset, load_fashion_mnist_dataset
from module.train import fit

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# 하이퍼파라미터 지정
EPOCH = 1
BATCH_SIZE = 256
LEARNING_RATE = 0.001
DATA_ROOT_DIR = "datasets"

## Data 준비

In [ ]:
train_loader = load_mnist_dataset(DATA_ROOT_DIR, BATCH_SIZE, True)
test_loader = load_mnist_dataset(DATA_ROOT_DIR, BATCH_SIZE, False)

## CNN 모델 정의

In [ ]:
# CNN - layer block
# ConvolutionLayer -> Activation -> Pooling Layer
# ConvolutionLayer -> Activation -> ConvolutionLayer -> Activation-> Pooling Layer

# ConvolutionLayer -> BatchNormalization -> Activation -> Dropout -> Pooling Layer

## 구조: filter 개수는 늘려주고(channel-depth) feature map의 size는 줄이는 방식으로 구성.
# depth: Conv2d,  size: MaxPool2d

class CNNModel(nn.Module):

    def __init__(self, dropout_rate=0.2):
        # 모델을 구성하는 Layer함수들을 초기화(객체 생성)
        super().__init__()
        # block 단위로 정의 - nn.Sequential()

        self.b1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,   # 입력 데이터의 channel 개수. 입력 tensor의 shape: (batch_size, channel, height, width) 
                                 # grayscale 이미지 -> channel수: 1, color: 3
                out_channels=32, # 필터의 개수 (output feature map의 개수)
                kernel_size=3,   # 필터의 크기 (3, 3)
                stride=1,        # 계산을 위하 이동 크기. 좌->우: 1칸씩, 상->하: 1칸 (default: 1)
                padding="same"   # 패팅 크기 (정수: 상하/좌우 동일할 패팅크기를 명시 - 0(default): 패딩추가 안함.)
                                 # "same": 입력 size와 동일한 size의 출력이 나오도록 알아서 패딩을 추가.
            ),
            nn.BatchNorm2d(32), # Conv: out_channels
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(
                kernel_size=2, # 값을 추출하는 영역 크기(2, 2) - default: 2
                stride=2       # 다음 값을 추출하기위해서 몇칸을 이동할지.(default: kernel_size)
            )
        )

        self.b2 = nn.Sequential(
            nn.Conv2d(
                in_channels=32, 
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding="same"
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.b3 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding="same"
            ),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )
        # 추론기(분류기) - Fully Conntected Layer(nn.Liear)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=4*4*128, out_features=10) # 최종 결과를 반환할 Layer. out_features=class개수
        )

    def forward(self, X):
        output = self.b1(X)
        output = self.b2(output)
        output = self.b3(output)
        output = self.classifier(output)
        return output

In [ ]:
model = CNNModel(dropout_rate=0.5)
model

In [ ]:
torchinfo.summary(model, (1, 1, 28, 28))

## Train

In [ ]:
# 모델
model = model.to(device)

# loss 함수
loss_fn = nn.CrossEntropyLoss() 
# 정답: One hot encoding처리, 추론값: Softmax 적용

# 옵티마이저
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
os.makedirs("saved_models", exist_ok=True)

In [ ]:
save_path = "saved_models/mnist_cnn_model.pt"
result = fit(
    train_loader, test_loader, model, loss_fn, optimizer, EPOCH,
    save_model_path=save_path,
    device=device, 
    mode="multi"
)

In [ ]:
len(test_loader)

In [ ]:
model = torch.load(save_path)

In [ ]:
# 최종 평가
from module.train import test_multi_classification
loss, acc = test_multi_classification(
    test_loader, model, loss_fn, device
)
loss, acc

In [ ]:
%pip install pillow

In [ ]:
## 정성적 평가 - 실제 image 파일로 확인
from PIL import Image
img = Image.open("test_img/num/eight.png")
type(img)

In [ ]:
from torchvision import transforms
from PIL import Image

def predict(path, model):
    img = Image.open(path)

    # color -> grayscale
    img = img.convert('L') # 'L': grayscale, "RGB": color

    # resize
    input_tensor = transforms.Resize((28, 28))(img)

    # PIL.Image -> torch.Tensor  변환, 정규화 (0 ~ 1)
    input_tensor = transforms.ToTensor()(input_tensor)

    # batch 축(dummy 축)을 추가.
    input_tensor = input_tensor.unsqueeze(dim=0)
    # print(type(input_tensor))
    # input_tensor.shape
    model.eval()
    model = model.to(device)
    
    with torch.no_grad():
        result = model(input_tensor)
    sm = nn.Softmax(dim=-1)
    result_proba = sm(result)
    final_result = result_proba.max(dim=-1)
    return {"class":final_result.indices[0], 
            "확률":final_result.values[0]}

In [ ]:
from glob import glob

img_path_list = glob("test_img/num/*.png")
for img_path in img_path_list:
    result = predict(img_path, model)
    print(f"{img_path}, 추론class: {result['class']}, 확률: {result['확률']}")